In [1]:
import pandas as pd
import numpy as np

In [2]:
# SAVR_TOBI_22sep-2okt.xlsx
# load impressions for 2022
df_02 = pd.read_csv('../datasets/impressions_02.csv')
df_03 = pd.read_csv('../datasets/impressions_03.csv')
df_q2 = pd.read_csv('../datasets/impressions_04_05_06.csv')
df_q3 = pd.read_csv('../datasets/impressions_07_08_09.csv')
df_q4 = pd.read_csv('../datasets/impressions_10_11_12.csv')
dfs = [df_02, df_03, df_q2, df_q3, df_q4]
impressions_df = pd.concat(dfs)

impressions_df

,id,hostname,placement_ids,is_fixated,gaze_valid
0,fa710e75-62cc-4489-8244-859320365bcd,gizmodo.com,"{""tag_id"":[],""bam_ad_slot"":["""","""",""""]}",False,False
1,8cd4bb75-b30c-49a8-9d55-e72b621b0ce0,gizmodo.com,"{""tag_id"":[],""bam_ad_slot"":["""","""",""""]}",False,False
2,ecd6ffa2-1a60-41fd-91f0-60656ae3dfa4,youtube.com,"{""tag_id"":[],""bam_ad_slot"":["""","""",""""]}",False,False
3,1c0a61de-9b56-41a2-b402-8d9280719919,tradera.com,"{""tag_id"":[],""bam_ad_slot"":["""","""",""""]}",False,False
4,c3c0b849-1adf-49d8-87bb-6860dc69adcd,tradera.com,"{""tag_id"":[],""bam_ad_slot"":["""","""",""""]}",False,False
...,...,...,...,...,...
5730217,7ef18388-e356-413c-830f-ba27bc48e422,youtube.com,"{""tag_id"":[],""bam_ad_slot"":["""","""",""""]}",False,False
5730218,75416ee9-3c04-4deb-9e25-4aeec3088480,youtube.com,"{""tag_id"":[],""bam_ad_slot"":["""","""",""""]}",False,False
5730219,834ecf80-ef5b-4d18-b20a-0a315c553588,youtube.com,"{""tag_id"":[],""bam_ad_slot"":["""","""",""""]}",False,False
5730220,bca44a62-380a-447d-bc07-70032f06fa36,youtube.com,"{""tag_id"":[],""bam_ad_slot"":["""","""",""""]}",False,False


In [3]:
import json

def extract_pid(placement_id):
    placement_id = json.loads(placement_id)
    bam_ad_slots = placement_id["bam_ad_slot"]
    # print(bam_ad_slots)
    final_bam_ad_slots = [bas for bas in bam_ad_slots if bas.strip() != '']
    ret = ''
    try:
        ret = placement_id['tag_id'][0]
    except:
        # print('.')
        ret = ','.join([bas for bas in final_bam_ad_slots])
    else:
        ret += ','.join([bas for bas in final_bam_ad_slots])
    return ret
    

def len_pids(placement_id):
    placement_id = json.loads(placement_id)
    bam_ad_slots = placement_id["bam_ad_slot"]
    final_bam_ad_slots = [bas for bas in bam_ad_slots if bas.strip() != '']
    return len(placement_id["tag_id"]) + len(final_bam_ad_slots)

def pid_type(pid):
    try:
        int(pid)
        return 'int'
    except:
        return 'str'

impressions_df['pid'] = impressions_df['placement_ids'].apply(extract_pid)
impressions_df['pid_len'] = impressions_df['placement_ids'].apply(len_pids)
impressions_df['pid_type'] = impressions_df['pid'].apply(pid_type)

In [4]:
# filter only impressions with int placement IDs
impressions_df = impressions_df[impressions_df['pid_type'] == 'int']
impressions_df.head()

,id,hostname,placement_ids,is_fixated,gaze_valid,pid,pid_len,pid_type
427,be031a8f-d75a-4af2-8f3a-32e2c661d11a,prisjakt.nu,"{""tag_id"":[""20334178""],""bam_ad_slot"":["""","""",""""]}",False,False,20334178,1,int
793,791fdc66-53af-4931-8a5f-49d52ef9a5b4,geeksforgeeks.org,"{""tag_id"":[""18677409""],""bam_ad_slot"":["""","""",""""]}",False,False,18677409,1,int
825,a48c49bb-0c82-4227-bd31-7b3daa2d0f97,aftonbladet.se,"{""tag_id"":[""19499443""],""bam_ad_slot"":["""","""",""""]}",False,True,19499443,1,int
826,f7f2b0a8-2615-44f2-83b2-93db796705f0,aftonbladet.se,"{""tag_id"":[""19499420""],""bam_ad_slot"":["""","""",""""]}",False,True,19499420,1,int
827,c2c85bb0-2a00-4188-9bc9-88f1e09d754f,aftonbladet.se,"{""tag_id"":[""19499441""],""bam_ad_slot"":["""","""",""""]}",False,True,19499441,1,int


In [5]:
# convert placement ids that can be converted to the int to be int, not string
# type(impressions_df.iloc[0]['pid'])     # check this, right now they're strings
impressions_df['pid'] = impressions_df['pid'].apply(lambda pid: int(pid))

/home/markopejic/clear/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
# run through predictions
import os
import json


def _get_pred_map():
    pred_map = dict()
    
    for i in range(1, 13):
        if i < 10:
            i = f'0{i}'
        for day in os.listdir(f'../predictions/2022/{i}'):
            for h in os.listdir(f'../predictions/2022/{i}/{day}'):
                if not h.endswith('.ndjson'):
                    continue
                with open(f'../predictions/2022/{i}/{day}/{h}')as f:
                    for line in f.readlines():
                        json_line = json.loads(line)
                        id_ = json_line['id']
                        pred_map[id_] = json_line['prediction']
    return pred_map
                            
pred_map = _get_pred_map()

In [7]:
def _get_final_fixation(gaze_valid, is_fixated, id_, pred_map):
    if gaze_valid:
        return is_fixated
    if id_ in pred_map:
        return pred_map[id_]
    return False

impressions_df['final_fixation'] = impressions_df.apply(lambda row: _get_final_fixation(row['gaze_valid'], row['is_fixated'], row['id'], pred_map), axis=1)
impressions_df

/home/markopejic/clear/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,id,hostname,placement_ids,is_fixated,gaze_valid,pid,pid_len,pid_type,final_fixation
427,be031a8f-d75a-4af2-8f3a-32e2c661d11a,prisjakt.nu,"{""tag_id"":[""20334178""],""bam_ad_slot"":["""","""",""""]}",False,False,20334178,1,int,True
793,791fdc66-53af-4931-8a5f-49d52ef9a5b4,geeksforgeeks.org,"{""tag_id"":[""18677409""],""bam_ad_slot"":["""","""",""""]}",False,False,18677409,1,int,False
825,a48c49bb-0c82-4227-bd31-7b3daa2d0f97,aftonbladet.se,"{""tag_id"":[""19499443""],""bam_ad_slot"":["""","""",""""]}",False,True,19499443,1,int,False
826,f7f2b0a8-2615-44f2-83b2-93db796705f0,aftonbladet.se,"{""tag_id"":[""19499420""],""bam_ad_slot"":["""","""",""""]}",False,True,19499420,1,int,False
827,c2c85bb0-2a00-4188-9bc9-88f1e09d754f,aftonbladet.se,"{""tag_id"":[""19499441""],""bam_ad_slot"":["""","""",""""]}",False,True,19499441,1,int,False
...,...,...,...,...,...,...,...,...,...
5730178,fffc6ec1-3fca-4082-a7e2-2706d87ea06e,prisjakt.nu,"{""tag_id"":[""20334189""],""bam_ad_slot"":["""","""",""""]}",False,False,20334189,1,int,False
5730179,79c391a7-69fe-4b6f-910c-882e77a749b2,dailyfitnesstip.com,"{""tag_id"":[""20602298""],""bam_ad_slot"":["""","""",""""]}",False,True,20602298,1,int,False
5730180,49018923-8fd0-451b-9dcc-dc047e905acd,dailyfitnesstip.com,"{""tag_id"":[""24405211""],""bam_ad_slot"":["""","""",""""]}",False,True,24405211,1,int,False
5730184,aa328d4f-557b-4284-812f-54453f6fd8da,aftonbladet.se,"{""tag_id"":[""19499442""],""bam_ad_slot"":["""","""",""""]}",False,False,19499442,1,int,False


In [8]:
# load schipsted dataset
df_client_schibsted = pd.read_excel('../datasets/SAVR_TOBI_22sep-2okt.xlsx')
df_client_schibsted

,LI Name,Placement Id,Clicks,Viewability Rate %
0,fullscreenscroll_mobil_aftonbladet,19499466,3048,0.880444
1,fullscreenscroll_mobil_aftonbladet,19499376,2566,0.932939
2,fullscreenscroll_mobil_aftonbladet,19499493,100,0.847743
3,fullscreenscroll_mobil_aftonbladet,19499398,116,0.898708
4,fullscreenscroll_svd/omni,19499584,2728,0.769116
5,fullscreenscroll_svd/omni,19499687,897,0.948325
6,fullscreenscroll_svd/omni,19499688,493,0.933987
7,fullscreenscroll_svd/omni,19499594,1488,0.969037
8,fullscreenscroll_svd/omni,19499743,444,0.951164
9,fullscreenscroll_svd/omni,19499722,322,0.751306


In [9]:
# filter only impressions with tele2 placement IDs
schibsted_placements = list(df_client_schibsted['Placement Id'].unique())
schibsted_impressions = impressions_df[impressions_df['pid'].isin(schibsted_placements)]
schibsted_impressions

,id,hostname,placement_ids,is_fixated,gaze_valid,pid,pid_len,pid_type,final_fixation
1494,4ef1c5ea-f0bf-48d6-9c3c-5fb8749cd082,aftonbladet.se,"{""tag_id"":[""19499440""],""bam_ad_slot"":["""","""",""""]}",False,True,19499440,1,int,False
1657,99d02d1d-6ff0-478b-b92c-088197c54dae,aftonbladet.se,"{""tag_id"":[""19499440""],""bam_ad_slot"":["""","""",""""]}",False,True,19499440,1,int,False
3331,2e7fd1d3-5c92-44c2-aeb2-f4f03e1f7736,aftonbladet.se,"{""tag_id"":[""19499440""],""bam_ad_slot"":["""","""",""""]}",False,False,19499440,1,int,True
4515,9b3307eb-0a63-4134-a334-7a4c956b20d1,aftonbladet.se,"{""tag_id"":[""19499440""],""bam_ad_slot"":["""","""",""""]}",True,True,19499440,1,int,True
7280,a6fcf87e-832c-4e04-8cf9-43f8582c5816,aftonbladet.se,"{""tag_id"":[""19499440""],""bam_ad_slot"":["""","""",""""]}",False,False,19499440,1,int,True
...,...,...,...,...,...,...,...,...,...
5724741,6f744c63-58d2-435e-a3e8-2923d2e512fa,aftonbladet.se,"{""tag_id"":[""19499440""],""bam_ad_slot"":["""","""",""""]}",False,False,19499440,1,int,True
5725723,4ce12d05-581e-4960-99f5-61b5e64e8544,aftonbladet.se,"{""tag_id"":[""19499440""],""bam_ad_slot"":["""","""",""""]}",False,False,19499440,1,int,True
5725724,80869fb8-4375-4725-9c81-fd65e4514615,aftonbladet.se,"{""tag_id"":[""19499440""],""bam_ad_slot"":["""","""",""""]}",False,False,19499440,1,int,True
5729857,d6adaf1f-5aaa-4150-b6bf-25d34149a7dc,svd.se,"{""tag_id"":[""19499711""],""bam_ad_slot"":["""","""",""""]}",False,True,19499711,1,int,False


In [10]:
print(f"{len(schibsted_impressions['pid'].value_counts())}/{len(schibsted_placements)} of placement IDs founded")

10/39 of placement IDs founded


In [15]:
found_pids = schibsted_impressions['pid'].unique()
not_found_pids = [pid for pid in schibsted_placements if pid not in found_pids]
print(found_pids)
print(not_found_pids)

[19499440 19499722 19499711 19499749 19499466 19499743 19499744 19499745
 19499489 19499740]
[19499376, 19499493, 19499398, 19499584, 19499687, 19499688, 19499594, 19499690, 19499394, 19499362, 19499693, 19499673, 14749497, 14749482, 20231334, 14749503, 14749488, 18031888, 14749494, 24657662, 18031886, 14749500, 21061757, 14749479, 24655304, 14749491, 14749485, 21061662, 23854743]


In [11]:
# groupby and aggregate
grouped_df = schibsted_impressions.groupby(['hostname', 'pid']).apply(
    lambda x: pd.Series(
        {
            'num_impressions' : x['id'].count(),
            'num_fixations': x.loc[x['final_fixation'],'id'].count()
        }
    )
)

In [12]:
grouped_df['ratio'] = round(grouped_df['num_fixations'] / grouped_df['num_impressions'], 4)
grouped_df['ratio_percent'] = round(grouped_df['ratio'] * 100, 2)
grouped_df

num_impressions  num_fixations   ratio  ratio_percent
hostname       pid                                                            
aftonbladet.se 19499440            16511           9126  0.5527          55.27
               19499466              451            194  0.4302          43.02
               19499489              265            162  0.6113          61.13
svd.se         19499711             2420            842  0.3479          34.79
               19499722             4915           3033  0.6171          61.71
               19499740               47             11  0.2340          23.40
               19499743              652            503  0.7715          77.15
               19499744              370            260  0.7027          70.27
               19499745              308            241  0.7825          78.25
               19499749              159             45  0.2830          28.30

In [13]:
def get_sample_size(num_impressions):
    if num_impressions <= 5:
        return "low"
    elif num_impressions > 5 and num_impressions <= 10:
        return "medium"
    else:
        return "high"
    
grouped_df['sample_size'] = grouped_df['num_impressions'].apply(lambda num_impressions: get_sample_size(num_impressions))
grouped_df

num_impressions  num_fixations   ratio  \
hostname       pid                                                
aftonbladet.se 19499440            16511           9126  0.5527   
               19499466              451            194  0.4302   
               19499489              265            162  0.6113   
svd.se         19499711             2420            842  0.3479   
               19499722             4915           3033  0.6171   
               19499740               47             11  0.2340   
               19499743              652            503  0.7715   
               19499744              370            260  0.7027   
               19499745              308            241  0.7825   
               19499749              159             45  0.2830   

                         ratio_percent sample_size  
hostname       pid                                  
aftonbladet.se 19499440          55.27        high  
               19499466          43.02        high  
               19499489          61.13        high  
svd.se         19499711          34.79        high  
               19499722          61.71        high  
               19499740          23.40        high  
               19499743          77.15        high  
               19499744          70.27        high  
               19499745          78.25        high  
               19499749          28.30        high

In [34]:
result_df1 = grouped_df.reset_index()
result_df1 = result_df1[['hostname', 'pid', 'ratio', 'ratio_percent', 'sample_size']]
result_df1.rename(columns={'pid': 'placement_id'}, inplace=True)
result_df1

,hostname,placement_id,ratio,ratio_percent,sample_size
0,aftonbladet.se,19499440,0.5527,55.27,high
1,aftonbladet.se,19499466,0.4302,43.02,high
2,aftonbladet.se,19499489,0.6113,61.13,high
3,svd.se,19499711,0.3479,34.79,high
4,svd.se,19499722,0.6171,61.71,high
5,svd.se,19499740,0.2340,23.40,high
6,svd.se,19499743,0.7715,77.15,high
7,svd.se,19499744,0.7027,70.27,high
8,svd.se,19499745,0.7825,78.25,high
9,svd.se,19499749,0.2830,28.30,high


# Try to find more placement IDs in our PID database

In [15]:
import pickle5 as pickle

pids = list()

for i in range(1, 13):
    if i < 10:
        i = f'0{i}'
    with open(f"beacon-to-impression/potential_pids_{i}_2022.pkl", "rb") as fh:
        data = pickle.load(fh)
        pids.append(data)
df_pids = pd.concat(pids)

In [16]:
df_pids

,potential_pid,surrounding,impression_id
0,133016,se,190450c2-2699-4fa3-8ff5-d77355bbe544
1,7086796,rm_eid,7e7ba672-4e86-46a4-92d0-8742a6d4230d
2,319672,text18,7e7ba672-4e86-46a4-92d0-8742a6d4230d
3,3196726,text18,7e7ba672-4e86-46a4-92d0-8742a6d4230d
4,319672,text18,7e7ba672-4e86-46a4-92d0-8742a6d4230d
...,...,...,...
1177532,2005575,dopb,af120965-03bf-4065-8757-af9926d1ad53
1177533,170642,dopb,af120965-03bf-4065-8757-af9926d1ad53
1177534,731103,bgipf,af120965-03bf-4065-8757-af9926d1ad53
1177535,3640501,nehu,af120965-03bf-4065-8757-af9926d1ad53


In [17]:
campaign_tags = ['cid', 'camid', 'campaign_id', 'buy', 'campid', 'campaignid', 'cmp', 'cpid',
                'kampanj', 'cpg_id', 'cpgid']
creative_tags = ['creid', 'creative', 'crid', 'creativeid', 'creative_id']
placement_tags = ['place', 'plid', 'pid', 'tag_id', 'tag', 'placementid', 'plc']
placement_group_tags = ['group']

def is_in_tag(item, tags):
    if len(item) < 3:
        return False
    for tag in tags:
        if tag in item:
            return True
    return False

In [18]:
df_pids['is_placement'] = df_pids['surrounding'].apply(lambda cell: is_in_tag(cell.lower(), placement_tags))

In [19]:
df_pids['is_group'] = df_pids['surrounding'].apply(lambda cell: is_in_tag(cell.lower(), placement_group_tags))

In [20]:
df_pids['is_campaign'] = df_pids['surrounding'].apply(lambda cell: is_in_tag(cell.lower(), campaign_tags))

In [21]:
df_pids = df_pids[(df_pids['is_placement']) & (~df_pids['is_group']) & (~df_pids['is_campaign'])]

In [22]:
df_pids = df_pids.drop_duplicates(['potential_pid', 'impression_id'])

In [23]:
df_pids

,potential_pid,surrounding,impression_id,is_placement,is_group,is_campaign
22,324460840,pid,7e7ba672-4e86-46a4-92d0-8742a6d4230d,True,False,False
103,316600711,pid,2ded50f2-eab5-4c9f-9dfd-52a3320c643c,True,False,False
133,19499424,place,2ded50f2-eab5-4c9f-9dfd-52a3320c643c,True,False,False
178,21067130,place,28bd2c51-fbae-487c-bad7-766f1d8f92fb,True,False,False
216,321746106,pid,139813b2-3fbd-40fa-902e-bf6083d2a344,True,False,False
...,...,...,...,...,...,...
1176531,6497154,pID,7e411642-86fe-48d2-9dd7-2e7cef9d85dc,True,False,False
1177031,19499424,TAG_ID,bc5fd7b5-474c-4c92-8060-22605c6a04e1,True,False,False
1177082,19404049,tag,ac656f57-808f-497b-883b-3978acf7db92,True,False,False
1177104,252522,252522TAG_ID,ac656f57-808f-497b-883b-3978acf7db92,True,False,False


In [24]:
schibsted_placements = list(schibsted_impressions['pid'].unique())
placement_ids = list(df_pids['potential_pid'].unique())

placements = list()
hostnames = list()
num_impressions = list()
num_fixations = list()
surrounding_list = list()
rates = list()

for pid in placement_ids:
    if pid not in schibsted_placements:
        continue
    df_pid_slice = df_pids[df_pids['potential_pid'] == pid]
    impressions = list(df_pid_slice['impression_id'].unique())
    surroundings = list(df_pid_slice['surrounding'].unique())
    
    df_imp_slice = impressions_df[impressions_df['id'].isin(impressions)]
    if df_imp_slice.empty:
        continue
    placements.append(pid)
    surrounding_list.append(surroundings)
    
    hostnames.append(df_imp_slice.iloc[0]['hostname'])
    num_imp = df_imp_slice.shape[0]
    num_fix = df_imp_slice[df_imp_slice['final_fixation']].shape[0]
    num_impressions.append(num_imp)
    num_fixations.append(num_fix)
    rates.append(num_fix/num_imp)

In [27]:
additional_pids_df = pd.DataFrame.from_dict({
    "hostname": hostnames,
    "placement_id": placements,
    "num_impressions": num_impressions,
    "num_fixations": num_fixations,
    "surroundings": surrounding_list,
    "ratio": rates,
    "ratio_percent": [f"{round(ratio*100, 2)}%" for ratio in rates]
})

In [28]:
additional_pids_df

,hostname,placement_id,num_impressions,num_fixations,surroundings,ratio,ratio_percent
0,aftonbladet.se,19499440,209,116,"[place, TAG_ID, tag_id]",0.555024,55.5%
1,svd.se,19499711,44,19,"[place, TAG_ID, tag_id]",0.431818,43.18%
2,svd.se,19499722,53,35,"[place, tag_id, TAG_ID]",0.660377,66.04%
3,aftonbladet.se,19499489,2,2,[place],1.000000,100.0%
4,svd.se,19499744,2,2,[TAG_ID],1.000000,100.0%
5,svd.se,19499743,2,2,[place],1.000000,100.0%
6,svd.se,19499745,3,3,[place],1.000000,100.0%


In [29]:
additional_pids_df['sample_size'] = additional_pids_df['num_impressions'].apply(lambda num_impressions: get_sample_size(num_impressions))
additional_pids_df

,hostname,placement_id,num_impressions,num_fixations,surroundings,ratio,ratio_percent,sample_size
0,aftonbladet.se,19499440,209,116,"[place, TAG_ID, tag_id]",0.555024,55.5%,high
1,svd.se,19499711,44,19,"[place, TAG_ID, tag_id]",0.431818,43.18%,high
2,svd.se,19499722,53,35,"[place, tag_id, TAG_ID]",0.660377,66.04%,high
3,aftonbladet.se,19499489,2,2,[place],1.000000,100.0%,low
4,svd.se,19499744,2,2,[TAG_ID],1.000000,100.0%,low
5,svd.se,19499743,2,2,[place],1.000000,100.0%,low
6,svd.se,19499745,3,3,[place],1.000000,100.0%,low


In [37]:
result_df2 = additional_pids_df[['hostname', 'placement_id', 'ratio', 'ratio_percent', 'sample_size']]

In [43]:
result_df = pd.concat([result_df1, result_df2])
result_df.drop_duplicates(['hostname', 'placement_id'], inplace=True)
result_df.sort_values(by=['hostname', 'placement_id'], inplace=True)
result_df

,hostname,placement_id,ratio,ratio_percent,sample_size
0,aftonbladet.se,19499440,0.5527,55.27,high
1,aftonbladet.se,19499466,0.4302,43.02,high
2,aftonbladet.se,19499489,0.6113,61.13,high
3,svd.se,19499711,0.3479,34.79,high
4,svd.se,19499722,0.6171,61.71,high
5,svd.se,19499740,0.2340,23.4,high
6,svd.se,19499743,0.7715,77.15,high
7,svd.se,19499744,0.7027,70.27,high
8,svd.se,19499745,0.7825,78.25,high
9,svd.se,19499749,0.2830,28.3,high


In [ ]:
result_df = result_df[['hostname', 'placement_id', 'ratio', 'ratio_percent', 'sample_size']]
result_df.to_csv('schibsted_report_2022.csv', index=False)